<a href="https://colab.research.google.com/github/michalis0/BigScaleAnalytics/blob/master/week2/solutions/bsa_lab_bigquery_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Week 2: Google BigQuery

## Walkthrough

### For Colab users

In [1]:
from google.colab import auth

auth.authenticate_user()
print("Authenticated")

Authenticated


#### For Jupyter users ONLY

Make sure to replace "PATH_TO_CREDENTIALS_FILE" with the *absolute* path to the JSON service account key (e.g., "C:/Users/John/credentials.json"). Here is the [documentation](https://cloud.google.com/iam/docs/creating-managing-service-account-keys).

In [ ]:
!pip install google-cloud-bigquery

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "PATH_TO_CREDENTIALS_FILE"

### Connect to BigQuery

In [2]:
import pandas as pd
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client(project="bsa-2021-sandbox-001122")  # CHANGE WITH YOUR OWN PROJECT ID

To run the cells below, you must first create a new dataset in BigQuery and upload the data from week 2 (`Teams.csv` and `Players.csv`) as tables. Since the CSV files are well-formed, you can automatically infer the schema when loading the data.

In [3]:
# Create a reference to the World Cup dataset
dataset_ref = client.dataset("world_cup", project="bsa-2021-sandbox-001122")  # CHANGE WITH YOUR OWN DATASET (NOT TABLE) NAME AND PROJECT ID

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [4]:
# List the tables in the dataset
tables = list(client.list_tables(dataset))
for table in tables:  
    print(table.table_id)

Players
Teams


In [5]:
# Create a reference to the "Teams" table
table_ref = dataset_ref.table("Teams")

# API request - fetch the table
teams_table = client.get_table(table_ref)

# Same for the "Players" table
table_ref = dataset_ref.table("Players")
players_table = client.get_table(table_ref)

In [6]:
# Display the schema of the "Teams" table 
players_table.schema

[SchemaField('surname', 'STRING', 'NULLABLE', None, ()),
 SchemaField('team', 'STRING', 'NULLABLE', None, ()),
 SchemaField('position', 'STRING', 'NULLABLE', None, ()),
 SchemaField('minutes', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('shots', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('passes', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('tackles', 'INTEGER', 'NULLABLE', None, ()),
 SchemaField('saves', 'INTEGER', 'NULLABLE', None, ())]

In [7]:
# Show a preview of the "Teams" table using pandas
client.list_rows(teams_table, max_results=5).to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Honduras,38,3,0,1,2,0,3,7,0
1,North Korea,105,3,0,0,3,1,12,2,0
2,Greece,13,3,1,0,2,2,5,5,0
3,Cameroon,19,3,0,0,3,2,5,5,0
4,New Zealand,78,3,0,3,0,2,2,6,0


### Example of queries

In [8]:
q1 = """
select *
from bsa-2021-sandbox-001122.world_cup.Teams
where ranking < 20
order by ranking
"""

query_job = client.query(q1)
query_job.to_dataframe()

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards
0,Brazil,1,5,3,1,1,9,4,7,2
1,Spain,2,6,5,0,1,7,2,3,0
2,Portugal,3,4,1,2,1,7,1,8,1
3,Netherlands,4,6,6,0,0,12,5,15,0
4,Italy,5,3,0,2,1,4,5,5,0
5,Germany,6,6,4,0,2,13,3,8,1
6,Argentina,7,5,4,0,1,10,6,7,0
7,England,8,4,1,2,1,3,5,6,0
8,France,9,3,0,1,2,1,4,6,1
9,Greece,13,3,1,0,2,2,5,5,0


In [9]:
# Join the two tables
q2 = """
select teams.*, players.surname, players.position, players.shots,
players.minutes, players.passes, players.tackles, players.saves
from bsa-2021-sandbox-001122.world_cup.Teams teams
join bsa-2021-sandbox-001122.world_cup.Players players
on teams.team = players.team
"""

query_job = client.query(q2)
query_job.to_dataframe().head(10)

,team,ranking,games,wins,draws,losses,goalsFor,goalsAgainst,yellowCards,redCards,surname,position,shots,minutes,passes,tackles,saves
0,Argentina,7,5,4,0,1,10,6,7,0,Milito,forward,0,91,33,0,0
1,Brazil,1,5,3,1,1,9,4,7,2,Grafite,forward,0,5,4,0,0
2,Germany,6,6,4,0,2,13,3,8,1,Kiessling,forward,0,7,5,0,0
3,Ghana,32,5,2,2,1,5,4,11,0,Adiyiah,forward,0,33,9,0,0
4,Ghana,32,5,2,2,1,5,4,11,0,Amoah,forward,0,11,4,1,0
5,Greece,13,3,1,0,2,2,5,5,0,Charisteas,forward,0,61,12,2,0
6,Honduras,38,3,0,1,2,0,3,7,0,Jerry Palacios,forward,0,84,19,0,0
7,Honduras,38,3,0,1,2,0,3,7,0,Walter Martinez,forward,0,125,43,1,0
8,Italy,5,3,0,2,1,4,5,5,0,Pazzini,forward,0,29,6,0,0
9,Ivory Coast,27,3,1,1,1,4,3,5,0,Doumbia,forward,0,11,2,0,0


You can try more queries on your own! (or re-use the queries you wrote in the other notebook of week 2 with the SQLite extension)

## Exercises

Let's build some queries for the `accidents_2016` data from the NHTSA traffic facilities dataset, which is publicly available on Google BigQuery.

### Setting up

In [10]:
# Create a reference to the traffic facilities dataset
traffic_ref = client.dataset("nhtsa_traffic_fatalities", project="bigquery-public-data")

# API request - fetch the dataset
traffic_dataset = client.get_dataset(traffic_ref)

In [11]:
# Construct a reference to the "accident_2016" table
accidents_ref = traffic_ref.table("accident_2016")

# API request - fetch the table
accidents_table = client.get_table(accidents_ref)

In [12]:
# Display schema
accidents_table.schema

[SchemaField('state_number', 'INTEGER', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC). For more info on the codes, please look at <C1/V1/D1/PC1/P1/NM1 State Number> section in the pdf: https://crashstats.nhtsa.dot.gov/Api/Public/ViewPublication/812315', ()),
 SchemaField('state_name', 'STRING', 'NULLABLE', 'This data element identifies the state in which the crash occurred. The codes are from the General Services Administration’s (GSA) publication of worldwide Geographic Location Codes (GLC).', ()),
 SchemaField('consecutive_number', 'INTEGER', 'NULLABLE', 'This data element is the unique case number assigned to each crash. It appears on each data file and is used to merge information from the data files together. xxxxxx Two Characters for State Code followed by Four Characters for Case Number', ()),
 SchemaField('number_of_vehicle_f

In [13]:
# Show a preview of the "accident_2016" table
client.list_rows(accidents_table, max_results=5).to_dataframe()

,state_number,state_name,consecutive_number,number_of_vehicle_forms_submitted_all,number_of_motor_vehicles_in_transport_mvit,number_of_parked_working_vehicles,number_of_forms_submitted_for_persons_not_in_motor_vehicles,number_of_persons_not_in_motor_vehicles_in_transport_mvit,number_of_persons_in_motor_vehicles_in_transport_mvit,number_of_forms_submitted_for_persons_in_motor_vehicles,...,minute_of_ems_arrival_at_hospital,related_factors_crash_level_1,related_factors_crash_level_1_name,related_factors_crash_level_2,related_factors_crash_level_2_name,related_factors_crash_level_3,related_factors_crash_level_3_name,number_of_fatalities,number_of_drunk_drivers,timestamp_of_crash
0,56,Wyoming,560099,1,1,0,0,0,1,1,...,88,0,None,0,None,0,None,1,0,2016-12-04 03:45:00+00:00
1,16,Idaho,160226,1,1,0,0,0,3,3,...,99,0,None,0,None,0,None,2,0,2016-12-16 16:33:00+00:00
2,5,Arkansas,50006,1,1,0,0,0,2,2,...,99,20,Police-Pursuit-Involved (Since 1994),0,None,0,None,1,0,2016-01-06 03:36:00+00:00
3,48,Texas,482027,1,1,0,0,0,5,5,...,35,0,None,0,None,0,None,1,0,2016-08-07 15:28:00+00:00
4,12,Florida,121570,1,1,0,0,0,5,5,...,99,0,None,0,None,0,None,1,0,2016-07-21 19:37:00+00:00


### Questions

You can access the table using `from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016`.

1) How many crashes happened in each month of 2016?

In [14]:
q3 = """
select month_of_crash, count(*) as count
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by month_of_crash
order by month_of_crash
"""

query_job_3 = client.query(q3)
query_job_3.to_dataframe()

,month_of_crash,count
0,1,2354
1,2,2426
2,3,2694
3,4,2713
4,5,3005
5,6,3025
6,7,3025
7,8,3134
8,9,3154
9,10,3287


2) Let's call the accidents involving more than 5 motor vehichles in transport involved as "massive_crash", and all others as "small crash". Return the number of each type of crash for each weather condition.

**Hint**: Use columns `atmospheric_conditions_1_name` and `number_of_motor_vehicles_in_transport_mvit`.

In [15]:
q4 = """
select
  atmospheric_conditions_1_name,
  (case when number_of_motor_vehicles_in_transport_mvit > 5 then "massive_crash"
  else "small_crash" end) as crash_type,
  count(*) as nb_crashes
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by atmospheric_conditions_1_name, crash_type
order by atmospheric_conditions_1_name
"""

query_job_4 = client.query(q4)
query_job_4.to_dataframe()

,atmospheric_conditions_1_name,crash_type,nb_crashes
0,"Blowing Sand, Soil, Dirt",small_crash,20
1,Blowing Snow,small_crash,10
2,Clear,small_crash,24893
3,Clear,massive_crash,63
4,Cloudy,small_crash,5112
5,Cloudy,massive_crash,14
6,"Fog, Smog, Smoke",small_crash,328
7,"Fog, Smog, Smoke",massive_crash,1
8,Freezing Rain or Drizzle,small_crash,17
9,Not Reported,small_crash,1552


3) Find the top 5 states with the highest number of drunk drivers involved in accidents.

In [16]:
q5 = """
select state_name, sum(number_of_drunk_drivers) as nb_drunk_drivers
from bigquery-public-data.nhtsa_traffic_fatalities.accident_2016
group by state_name
order by nb_drunk_drivers desc
limit 5
"""

query_job_5 = client.query(q5)
query_job_5.to_dataframe()

,state_name,nb_drunk_drivers
0,California,1052
1,Texas,918
2,Florida,665
3,North Carolina,399
4,Ohio,365


4) Let's find out the states where using a cell phone is a major cause of accidents. Return the first 5 state names along with the number of crashes where the driver was distracted "while manipulating cellular phone".

**Hint**: You need to join the `distract_2016` and `accident_2016` tables.

In [17]:
q6 = """
select
  count(distract.driver_distracted_by) as nb_accidents,
  accident.state_name
from bigquery-public-data.nhtsa_traffic_fatalities.distract_2016 distract
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on distract.consecutive_number = accident.consecutive_number
where driver_distracted_by = 6
group by state_name
order by nb_accidents desc
limit 5
"""

query_job_6 = client.query(q6)
query_job_6.to_dataframe()

,nb_accidents,state_name
0,18,Texas
1,9,Missouri
2,9,California
3,8,North Carolina
4,8,Georgia


Another possible solution:

In [18]:
q6_1 = """
with
  join_acc_dist as (
    select distract.driver_distracted_by, accident.state_name
    from bigquery-public-data.nhtsa_traffic_fatalities.distract_2016 distract
    join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
    on distract.consecutive_number = accident.consecutive_number
    )
select count(driver_distracted_by) as nb_accidents, state_name from join_acc_dist
where driver_distracted_by = 6
group by state_name
order by nb_accidents desc
limit 5
"""

query_job_6_1 = client.query(q6_1)
query_job_6_1.to_dataframe()

,nb_accidents,state_name
0,18,Texas
1,9,Missouri
2,9,California
3,8,North Carolina
4,8,Georgia


5) Sometimes, a "live animal" is a reason that a driver maneuvers and causes an accident. In which months of the year do such accidents happen the most? Return the month and the number of accidents. 

**Hint**: You need to join the `maneuver_2016` and `accident_2016` tables. 

In [19]:
q7 = """
select
  accident.month_of_crash,
  count(maneuver.driver_maneuvered_to_avoid) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on maneuver.consecutive_number = maneuver.consecutive_number
where driver_maneuvered_to_avoid = 3
group by month_of_crash
order by nb_accidents desc
"""

query_job_7 = client.query(q7)
query_job_7.to_dataframe()

,month_of_crash,nb_accidents
0,10,384579
1,9,369018
2,8,366678
3,11,355797
4,6,353925
5,7,353925
6,5,351585
7,12,338130
8,4,317421
9,3,315198


6) In the state of Alaska, in which months do "poor road conditions" cause the most accidents? Return the month and the number of accidents. 

**Hint**: Again, you will need the `maneuver_2016` table. 

In [20]:
q8 = """
select
  accident.month_of_crash,
  count(maneuver.driver_maneuvered_to_avoid) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on maneuver.consecutive_number = maneuver.consecutive_number
where driver_maneuvered_to_avoid = 3
  and accident.state_name = "Alaska"
group by month_of_crash
order by nb_accidents desc
"""

query_job_8 = client.query(q8)
query_job_8.to_dataframe()

,month_of_crash,nb_accidents
0,7,1053
1,11,1053
2,8,936
3,6,936
4,4,819
5,10,819
6,1,702
7,5,702
8,9,702
9,12,702


7) Sometimes, a factor of the accident is that the vehicle has a problem with the "brake system". Filter the accidents with this factor and return the `driver_maneuvered_to_avoid_name` column along with the related number of accidents.
 
**Hint**: You need to join the `maneuver_2016` and `factor_2016` tables.

In [21]:
q9 = """
select
  maneuver.driver_maneuvered_to_avoid_name,
  count(maneuver.consecutive_number) as num_incidents
from bigquery-public-data.nhtsa_traffic_fatalities.maneuver_2016 maneuver
join bigquery-public-data.nhtsa_traffic_fatalities.factor_2016 factor
on maneuver.consecutive_number = factor.consecutive_number
where factor.contributing_circumstances_motor_vehicle = 2
group by driver_maneuvered_to_avoid_name
order by num_incidents desc
"""

query_job_9 = client.query(q9)
query_job_9.to_dataframe()

,driver_maneuvered_to_avoid_name,num_incidents
0,Driver Did Not Maneuver To Avoid,97
1,Not Reported,93
2,Motor Vehicle,25
3,Phantom/Non-Contact Motor Vehicle,5
4,No Driver Present/Unknown if Driver Present,3
5,Live Animal,1


8) Sometimes, a non-motorist is involved in a crash. For each state, find the number of crashes in which an "inattentive" person was involved. Return the state name and number of crashes.

**Hint**: You need to join the `nmcrash_2016` and `accident_2016` tables.

In [22]:
q10 = """
select
  accident.state_name,
  count(nmcrash.consecutive_number) as nb_accidents
from bigquery-public-data.nhtsa_traffic_fatalities.nmcrash_2016 nmcrash
join bigquery-public-data.nhtsa_traffic_fatalities.accident_2016 accident
on nmcrash.consecutive_number = accident.consecutive_number
where nmcrash.non_motorist_contributing_circumstances = 6
group by state_name
order by nb_accidents desc
"""

query_job_10 = client.query(q10)
query_job_10.to_dataframe()

,state_name,nb_accidents
0,Louisiana,17
1,Georgia,16
2,North Carolina,14
3,Missouri,12
4,Washington,9
5,California,8
6,Ohio,7
7,Texas,7
8,New Jersey,6
9,Florida,4
